## Mingu Kim- 5/22/17

# Exploration of Structured Bandit

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.stats import norm
import seaborn as sns
sns.set_style("whitegrid")

$$r_t \sim N(\theta(s_t, a_t), \tau^{-1})$$

In [4]:
def sample_r(theta_val):
    return np.random.normal(theta_val, tau ** (-1))

$$\theta(s,a) \sim N(0, \lambda_0^{-1}(s,a))$$

In [5]:
def theta(s, a):
    return np.random.normal(0, lambda_1(s,a) ** (-1))

**Here we assume precisions are the same for all states, actions **
$$\lambda_1(s,a) = \lambda_0$$

In [6]:
def lambda_1(s,a):
    return lambda0

$$\lambda_t(s_t,a_t) = \lambda_{t-1}(s_t,a_t) + \tau$$
$$\lambda_t(s_t, a_t) = \lambda_1(s,a) + (t-1) \tau$$

In [7]:
def lambda_t(t, s, a):
    return 0.1 * (t-1) * tau + lambda_1(s, a)

$$\eta_t = \frac{\tau}{\tau + \lambda_t(s_t,a_t)}$$ 

In [8]:
def eta(t, s, a):
    return tau / (tau + lambda_t(t, s, a))

$$\hat{\theta}_t(s_t,a_t) =\hat{\theta}_{t-1} (s_t,a_t) + \alpha_s \eta_t [r_{t-1} - \hat{\theta}_{t-1}(s_t,a_t)] $$
We also know that $$\hat{\theta}_1(s_t,a_t) = 0.$$

** I used a dictionary to implement my posterior means, so that we can flexibly add (s,a) pairs as needed**

In [9]:
def theta_t(thetas, r, s, a, t):
    updated_thetas = thetas.copy()
    if (s,a) not in thetas:
        updated_thetas[(s, a)] = eta(1, s, a) * r
    else:
        updated_thetas[(s, a)] \
        = updated_thetas[(s, a)] + eta(t, s, a) * (r - updated_thetas[(s, a)])
    return updated_thetas

$$P(r_t \mid s_t, a_t) = N(r_t; \hat{\theta}_t (s_t, a_t), \lambda_t(s_t, a_t)^{-1} + \tau^{-1})$$

In [58]:
def likelihood(r, t, s, a, thetas):
    return norm(thetas.get((s, a), 0), tau ** (-1) + lambda_t(t, s, a) ** (-1)).pdf(r) 

$$
  P(s_t = k \mid \textbf{s}_{1:t-1}) =
  \begin{cases}
   \frac{C_k + \omega \mathbb{I}[s_t = s_{t-1}]}{t+\alpha - 1} & \text{if $k \leq K$} \\
   \frac{\alpha}{t+\alpha-1} & \text{if $k = K_1$} 
  \end{cases}
$$
where $\mathbb{I}$[·] = 1 when its argument is true (0 otherwise), $C_k$ is the number of previous trials state $k$
was sampled, and $K$ is the total number of distinct previously sampled states. The concentration
parameter $\alpha > 0$ controls the propensity for sampling new states; the expected number of distinct
states after $t$ trials is $\alpha \ln t$. When $\alpha = 0$, only a single state will be sampled for all trials, and when
α approaches infinity, each trial will be associated with a distinct state. The parameter $\omega$ specifies
the “stickiness” or autocorrelation of states: when stickiness is high, consecutive trials will tend to
sample the same state.

In [11]:
#Takes k, $\omega$, $\alpha$, and visited states
def prior_prob(k,w, alpha, s_list):
    K = len(set(s_list))
    if k <= K - 1:
        return (s_list.count(k) + w * int(k == s_list[-1])) / (len(s_list) + alpha)
    elif k == K:
        return alpha / (len(s_list) + alpha)
    else:
        return "ERROR"

In [12]:
# Gives normalized probabilities of visiting each state
def prior_norm(w, alpha, s_list):
    K = len(set(s_list))
    probs = np.zeros(K + 1)
    for k in range(K+1):
        probs[k] = prior_prob(k,w,alpha, s_list)
    return np.array([float(i)/sum(probs) for i in probs])

$$\DeclareMathOperator*{\argmax}{argmax}$$
$$P(s_t \mid \textbf{a}_{1:t-1}, \textbf{r}_{1:t-1}) \approx P(s_t \mid \textbf{a}_{1:t-1}, \textbf{r}_{1:t-1}, \hat{\textbf{s}}_{1:t-1}), $$

where $\hat{s}_t$ is defined recursively as follows:

$$\hat{s}_t = \argmax_{s_t} P(r_t \mid s_t, \textbf{a}_{1:t}, \textbf{r}_{1:t-1}, \hat{\textbf{s}}_{1:t-1}) P(s_t \mid \hat{\textbf{s}}_{1:t-1})$$

In [13]:
def s_hat(s_list, t, r, a, w, alpha, thetas):
    priors = prior_norm(w, alpha, s_list)
    #print(priors)
    s_hats = np.array([likelihood(r, t, s, a, thetas) * priors[s] for s in range(len(priors))])
    return np.argmax(s_hats)

Not sure if I wrote it right, but the second one is true to recursive definition, while first one is a shortcut

In [14]:
def prior_map_est(s_list, reward_list, action_list, w, alpha, thetas):
    if len(reward_list) == 0:
        return [s_list, [1]]
    s_list.append(s_hat(s_list, i, reward_list[-1], action_list[-1], w, alpha, thetas))
    return [s_list, prior_norm(w, alpha, s_list)]

In [15]:
def prior_map_est(reward_list, action_list, w, alpha, thetas):
    s_list = [0] * (len(reward_list) + 1)
    for i in range(len(reward_list)):
        s_list[i+1] = s_hat(s_list[:i+1], i, reward_list[i], action_list[i], w, alpha, thetas)
    return prior_norm(w, alpha, s_list)

$$b_t(s) \propto P(r_t \mid s_t = s, a_t) P(s_t = s \mid \textbf{a}_{1:t-1}, \textbf{r}_{1:t-1})$$

In [50]:
def posterior(prior, state, last_action, reward_list, w, alpha, thetas):   
    if state > len(prior):
        return 0
    return likelihood(reward_list[-1], len(reward_list), state, last_action, thetas) * prior[state]

In [49]:
def posterior_norm(prior, last_action, reward_list, w, alpha, thetas):   
    probs = np.zeros(len(prior))
    for k in range(len(prior)):
        probs[k] = posterior(prior, k, last_action, reward_list, w, alpha, thetas)
    return np.array([float(i)/sum(probs) for i in probs])

$$ \mu_t(a) = \sum_s b_t(s) \hat{\theta}_t(s,a)$$

In [52]:
def mu_t(prior,action, reward_list, w, alpha, thetas):
    exp_reward = 0
    for state in range(len(prior)):
        exp_reward += posterior(prior, state, action, reward_list, w, alpha, thetas) \
        * thetas.get((state, action), 0)
    return exp_reward

$$\pi_t(s) = e^{\beta \mu_{t-1} (a)}$$

In [53]:
def softmax_action(prior, actions, action_list, reward_list, w, alpha, thetas, beta):
    if len(action_list) == 0:
        return np.random.choice(len(actions))
    action_vals = []
    for action in actions:
        action_vals.append(np.exp(beta * mu_t(prior, action, reward_list, w, alpha, thetas)))
    actions_normed = np.array(action_vals) / sum(action_vals)
    return np.random.choice(len(actions), 1, p=actions_normed)[0]

$$Q_t(a) = \mu_{t-1} (a) + \beta^{-1} \sigma_{t-1}(a)$$

$$\sigma_t(a) = \sqrt{\sum_s b_t(s) [(\hat{\theta}_t(s,a) - \mu_t(s,a))^2 + 1/\lambda_t(s,a)]}$$

$$\pi_t(a) = (1-\epsilon) \mathbb{I}[a=a^{*}] + \frac{\epsilon}{A} (1-\mathbb{I}[a=a^{*}]))$$


In [20]:
# def update_thetas(thetas, r, a, prior):
#     updated_thetas = thetas.copy()
#     s = np.random.choice(len(prior), p = prior)
#     try:
#         updated_thetas[(s, a)] \
#         = [updated_thetas[(s, a)][0] + eta(updated_thetas[(s, a)][1], s, a) * (r - updated_thetas[(s, a)][0]), updated_thetas[(s, a)][1] + 1]
#     except KeyError:
#         updated_thetas[(s, a)] = [eta(1, s, a) * r, 2]
#     return updated_thetas

In [21]:
def update_thetas(thetas, r, a, posterior, t):
    updated_thetas = thetas.copy()
    for s in range(len(posterior)):
        try:
            updated_thetas[(s, a)] \
            = updated_thetas[(s, a)] + posterior[s] * eta(t, s, a) * (r - updated_thetas[(s, a)])
        except KeyError:
            updated_thetas[(s, a)] = posterior[s] * eta(1, s, a) * r
    return updated_thetas

In [54]:
def UCB_action(prior, actions, action_list, reward_list, w, alpha, thetas, beta, epsilon):
    if len(action_list) == 0:
        return np.random.choice(len(actions))
    action_vals = []
    Q = []
    for action in actions:
        sigma = 0
        for s in range(len(prior)):
            sigma+= posterior(prior, s, action_list[-1], reward_list, w, alpha, thetas) \
            * ((thetas.get((s, action), 0) - mu_t(prior, action, reward_list, w, alpha, thetas))**2\
            + 1 / lambda_t(len(reward_list), s, action))
        if sigma > 0:
            sigma = np.sqrt(sigma)
        else:
            sigma = 0
        Q.append(mu_t(prior, action, reward_list, w, alpha, thetas) + beta ** (-1)* sigma)
    best_action = np.argmax(Q)
    for action in actions:
        if action == best_action:
            action_vals.append(1 - epsilon)
        else:
            action_vals.append(epsilon / len(actions))
    actions_normed = np.array(action_vals) / sum(action_vals)
    return np.random.choice(len(actions), 1, p=actions_normed)[0]

# Using UCB Method

In [241]:
rewards

array([[7, 9, 4],
       [5, 5, 7],
       [0, 5, 1],
       [0, 5, 3],
       [4, 6, 5],
       [6, 4, 8],
       [9, 4, 2],
       [2, 9, 7],
       [6, 3, 9],
       [2, 4, 6]])

In [250]:
np.set_printoptions(suppress=True)
past_actions = []
past_rewards = []
exp_rewards = {}
tau = 2
lambda0 = 1
w = 0.7
alpha = 1.5
beta = 5
# total_actions = range(2)
# rewards = [[10, 0], [0, 10], [5, 0], [0, 5],[10, 0], [0, 10], [5, 0], [0, 5],[10, 0], [0, 10], [5, 0], [0, 5]]
total_actions = range(3)
#rewards = [[10, 0, 0], [0, 10, 0], [0, 0, 10],[10, 0, 0], [0, 10, 0], [0, 0, 10], [10, 0, 0], [0, 10, 0],[10, 0, 0], [0, 10, 0], [0, 0, 10]]
rewards = np.random.randint(10, size=(10, 3))
states = [0]
s_list = [0]
epsilon = 1
inc = 0.03
post = [1]
random_bot_rewards = []
simple_bot_actions = np.zeros(len(total_actions))
simple_bot_rewards = []
for i in range(100):
    curr_action = UCB_action(post, total_actions, past_actions, past_rewards, w, alpha, exp_rewards, beta, epsilon)
    curr_reward = np.random.normal(rewards[states[-1]][curr_action], 1.0/tau)
    #curr_reward = rewards[states[-1]][curr_action]
    random_bot_rewards.append(np.random.normal(rewards[states[-1]][np.random.randint(len(total_actions))], 1.0/tau))
    simple_bot_rewards.append(np.random.normal(rewards[states[-1]] \
                              [np.argmax(simple_bot_actions)], 1.0/tau))
    simple_bot_actions[np.argmax(simple_bot_actions)] = simple_bot_rewards[-1]
    #print(curr_action, curr_reward)
    past_actions.append(curr_action)
    past_rewards.append(curr_reward)
    prior = prior_map_est(past_rewards[:-1], past_actions[:-1], w, alpha, exp_rewards)
    #prior = prior_map_est(past_rewards, past_actions, w, alpha, exp_rewards)
    post = posterior_norm(prior, past_actions[-1], past_rewards, w, alpha, exp_rewards)
    #print(post)
    exp_rewards = update_thetas(exp_rewards, curr_reward, curr_action, post, len(past_actions))
    #print(exp_rewards)
    #print()
    state_probs = prior_norm(w, alpha, states)
    #print('state_probs: ', state_probs)
    states.append(np.random.choice(len(state_probs), p=state_probs))
    if epsilon > inc:
        epsilon -= inc
# ax = sns.countplot(past_actions)
# ax.set(xlabel='Arm selected', ylabel='Number of times arm was selected', title='Arm Selection with UCB Method')
# for p in ax.patches:
#     ax.annotate('r = ' + str(rewards[int(p.get_x()+0.4)]), (p.get_x() + 0.23, p.get_height() +0.25))
# plt.show()

In [251]:
# The current agent
sum(past_rewards)

468.1745106770399

In [252]:
# Random actions
sum(random_bot_rewards)

376.81694282601603

In [253]:
# Remembers the last reward for each arm, picks the arm that had the biggest last reward
sum(simple_bot_rewards)

383.32151108773235

**Calculated number of times each state has been visited**

In [65]:
from collections import Counter
Counter(states)

Counter({0: 10, 1: 9, 2: 1, 3: 2, 4: 6, 5: 2, 6: 1})

In [292]:
exp_rewards

{(0, 0): 9.9967600269665322,
 (0, 1): 2.9415907287332725,
 (1, 0): 0.89142927104979397,
 (1, 1): 0.13193251578597834,
 (2, 0): 0.0049081323519856749,
 (2, 1): 0.43164144830557533,
 (3, 0): 1.9263951042410362e-41,
 (3, 1): 0.054011670533130676}

In [55]:
prior_map_est([0, 0, 0],[0, 5, 5], [0, 1, 1], w, alpha, exp_rewards)

[[0, 0, 0, 0], array([ 0.90566038,  0.09433962])]

In [39]:
lambda0=0.5
tau ** (-1) + lambda_t(2, 0, 1) ** (-1)

1.9285714285714286